In [ ]:
def generate_new_points(geom: gpd.GeoSeries) -> dict[int, set[shapely.Point]]:
    points_to_add: dict[int, set[shapely.Point]] = dict()
    g1: shapely.Polygon
    g2: shapely.Polygon
    for i1, geometry_1 in enumerate(geom):
        for x, y in zip(*geometry_1.exterior.coords.xy):
            point = shapely.Point(x, y)
            for i2, geometry_2 in enumerate(geom):
                if i2 == i1:  # matching itsself does not make sense
                    continue
                distance = geometry_2.distance(point)
                if distance > 0.125:
                    continue  # semi arbitrary cutoff after which merging does not make sense
                # insert point perpendicular
                points_to_add.setdefault(i2, set())
                for perpendicular_point in nearest_points(geometry_2, point):
                    points_to_add[i2].add(perpendicular_point)

    return points_to_add


hallucinated_points = generate_new_points(spaces.geometry)

In [ ]:
def visualise_hallucinated_points():
    points_to_add_list = list(itertools.chain.from_iterable(hallucinated_points.values()))
    hallucinated_point_df = gpd.GeoDataFrame(geometry=list(points_to_add_list), crs=df.crs)
    ax = spaces.plot(color="white", edgecolor="black", figsize=(50, 50))
    ax.set_axis_off()
    hallucinated_point_df.plot(ax=ax, marker="x", color="pink", markersize=100)


visualise_hallucinated_points()

In [ ]:
def apply_hallucinated_points():
    geom = []
    for i, g in enumerate(spaces["geometry"]):
        if i not in hallucinated_points:
            geom.append(g)
            continue

        if points := hallucinated_points[i]:
            # from https://gis.stackexchange.com/questions/188594/how-can-i-add-points-to-a-linestring-in-shapely
            ring = shapely.LineString(points)
            union = ring.union(g)
            g = polygonize(union)[0]
        geom.append(g)
    spaces["geometry"] = geom


apply_hallucinated_points()
visualise_points()

In [ ]:
def merge_points(point: shapely.Point, possible_points: set[shapely.Point]) -> shapely.Point:
    assert len(possible_points) >= 1
    assert shapely.Point(point)
    assert all([shapely.Point(p) for p in possible_points])

    new_geometry = [point]
    while True:
        geom = shapely.MultiPoint(new_geometry)
        matching_points = set()
        for p2 in possible_points:
            if geom.distance(p2) < 0.125:
                matching_points.add(p2)
        if not matching_points:
            return geom.centroid
        possible_points.difference_update(matching_points)
        new_geometry.extend(matching_points)


def move_geometries():
    geom = []
    for g1 in spaces["geometry"]:
        points = [shapely.Point(x, y) for (x, y) in zip(*g1.exterior.coords.xy)]
        for i, point in enumerate(points):
            possible_points = set()
            for g2 in spaces["geometry"]:
                if g2.distance(point) < 3:
                    for x, y in zip(*g2.exterior.coords.xy):
                        possible_points.add(shapely.Point(x, y))
            if possible_points:
                points[i] = merge_points(point, possible_points)
        geom.append(shapely.Polygon(points))
    spaces["geometry"] = geom


move_geometries()
visualise_points()

In [ ]:
simplify_shapes()
visualise_points()